In [2]:
import pandas as pd
import os
import numpy as np

In [3]:
df = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/raw_data/data_mc.csv', header=None)

In [6]:
df_test = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/raw_data/test.csv', header=None)

In [4]:
df.head()

,0
0,"I'm not a huge mcds lover, but I've been to be..."
1,Terrible customer service. Î¾I came in at 9:30...
2,"First they ""lost"" my order, actually they gave..."
3,I see I'm not the only one giving 1 star. Only...
4,"Well, it's McDonald's, so you know what the fo..."


In [7]:
df_test = df_test.rename({0: 'label', 1: 'text'}, axis = 1)

In [8]:
df = df.rename({0: 'label', 1: 'text'}, axis = 1)

In [9]:
df_test

,label,text
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...
...,...,...
37995,1,If I could give 0...I would. Don't do it.
37996,2,Items Selected:\nChocolate Cinnamon Horn\nSmal...
37997,1,Expensive lunch meals. Fried pickles were goo...
37998,1,Highly overpriced and food was cold. Our waitr...


In [10]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(data, percentage_of_sentences=None):
    
    train_sentences = data['text']
    y_train = data['label']
    
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
    X_train = [text_to_word_sequence(_) for _ in train_sentences]
    
  
    return X_train, y_train



In [11]:
X_train, y_train = load_data(df, percentage_of_sentences=10)

In [12]:
X_test, y_test = load_data(df_test, percentage_of_sentences=10)

In [13]:
len_train = int(10/100*len(df_test['text']))
X_test = df_test['text']
X_test = X_test[:len_train]

In [14]:
X_test

0       Contrary to other reviews, I have zero complai...
1       Last summer I had an appointment to get new ti...
2       Friendly staff, same starbucks fair you get an...
3       The food is good. Unfortunately the service is...
4       Even when we didn't have a car Filene's Baseme...
                              ...                        
3795    Came here on a Tues night at 6pm. Warning. If ...
3796    I've been here since my teeny bopper years and...
3797    Friendly staff and quick service for auto tint...
3798    Please be warned! Do not patronize Home Fabric...
3799    I only went for an event in the back room. As ...
Name: text, Length: 3800, dtype: object

In [15]:
y_test_adj = y_test -1

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
    
# We apply the tokenization to the train and test set
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)


X_train_pad = pad_sequences(X_train_token, maxlen=50, dtype='float32', padding="post")
X_test_pad = pad_sequences(X_test_token, maxlen=50, dtype='float32', padding="post")

vocab_size = len(tokenizer.word_index)

In [17]:
from tensorflow.keras import Sequential, layers

def init_cnn_model(vocab_size):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size + 1, output_dim=20, mask_zero=True, input_length=50))
    model.add(layers.Conv1D(32, 3))
    model.add(layers.Flatten())
    model.add(layers.Dense(50, activation="relu"))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_cnn = init_cnn_model(vocab_size)

In [18]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 20)            1378040   
                                                                 
 conv1d (Conv1D)             (None, 48, 32)            1952      
                                                                 
 flatten (Flatten)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 50)                76850     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1,456,893
Trainable params: 1,456,893
Non-trainable params: 0
_________________________________________________________________


In [19]:
y_train_adj = y_train -1

In [20]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad, y_train_adj, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


Epoch 1/20


2022-12-03 22:27:09.232442: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1225/1225 [==============================] - 7s 6ms/step - loss: 0.3580 - accuracy: 0.8371 - val_loss: 0.2827 - val_accuracy: 0.8827
Epoch 2/20
1225/1225 [==============================] - 7s 6ms/step - loss: 0.1992 - accuracy: 0.9201 - val_loss: 0.3171 - val_accuracy: 0.8716
Epoch 3/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.0810 - accuracy: 0.9699 - val_loss: 0.4644 - val_accuracy: 0.8505
Epoch 4/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.0257 - accuracy: 0.9914 - val_loss: 0.6262 - val_accuracy: 0.8542
Epoch 5/20
1225/1225 [==============================] - 7s 5ms/step - loss: 0.0129 - accuracy: 0.9955 - val_loss: 0.6918 - val_accuracy: 0.8411
Epoch 6/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.0105 - accuracy: 0.9961 - val_loss: 0.9219 - val_accuracy: 0.8485


In [21]:
res = model_cnn.evaluate(X_test_pad, y_test_adj)

119/119 [==============================] - 0s 633us/step - loss: 0.2794 - accuracy: 0.8829


In [22]:
y_pred = model_cnn.predict(X_test_pad)

119/119 [==============================] - 0s 626us/step


In [23]:
y_pred

array([[0.3370884 ],
       [0.01962742],
       [0.96878624],
       ...,
       [0.03655507],
       [0.04790182],
       [0.28231007]], dtype=float32)

In [24]:
y_pred = np.round(y_pred)

In [25]:
y_pred = y_pred+1

In [26]:
y_pred = pd.DataFrame(y_pred)

In [27]:
y_pred

,0
0,1.0
1,1.0
2,2.0
3,1.0
4,1.0
...,...
3795,2.0
3796,2.0
3797,1.0
3798,1.0


In [28]:
X_test = pd.DataFrame(X_test)

In [29]:
X_test['label_pred'] = y_pred

In [30]:
X_test

,text,label_pred
0,"Contrary to other reviews, I have zero complai...",1.0
1,Last summer I had an appointment to get new ti...,1.0
2,"Friendly staff, same starbucks fair you get an...",2.0
3,The food is good. Unfortunately the service is...,1.0
4,Even when we didn't have a car Filene's Baseme...,1.0
...,...,...
3795,Came here on a Tues night at 6pm. Warning. If ...,2.0
3796,I've been here since my teeny bopper years and...,2.0
3797,Friendly staff and quick service for auto tint...,1.0
3798,Please be warned! Do not patronize Home Fabric...,1.0


In [31]:
X_test['sentiment'] = X_test['label_pred'].apply(lambda x: 'negative' if x == 1.0 else 'positive')

In [32]:
X_test

,text,label_pred,sentiment
0,"Contrary to other reviews, I have zero complai...",1.0,negative
1,Last summer I had an appointment to get new ti...,1.0,negative
2,"Friendly staff, same starbucks fair you get an...",2.0,positive
3,The food is good. Unfortunately the service is...,1.0,negative
4,Even when we didn't have a car Filene's Baseme...,1.0,negative
...,...,...,...
3795,Came here on a Tues night at 6pm. Warning. If ...,2.0,positive
3796,I've been here since my teeny bopper years and...,2.0,positive
3797,Friendly staff and quick service for auto tint...,1.0,negative
3798,Please be warned! Do not patronize Home Fabric...,1.0,negative


In [34]:
X_test.drop(columns='label_pred')

,text,sentiment
0,"Contrary to other reviews, I have zero complai...",negative
1,Last summer I had an appointment to get new ti...,negative
2,"Friendly staff, same starbucks fair you get an...",positive
3,The food is good. Unfortunately the service is...,negative
4,Even when we didn't have a car Filene's Baseme...,negative
...,...,...
3795,Came here on a Tues night at 6pm. Warning. If ...,positive
3796,I've been here since my teeny bopper years and...,positive
3797,Friendly staff and quick service for auto tint...,negative
3798,Please be warned! Do not patronize Home Fabric...,negative


In [35]:
X_test.sentiment.value_counts(normalize=True)

negative    0.545263
positive    0.454737
Name: sentiment, dtype: float64

In [37]:
model_cnn.save('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/CNN_model')

INFO:tensorflow:Assets written to: /Users/marikhomeriki/code/marikhomeriki/product_review_analysis/CNN_model/assets


INFO:tensorflow:Assets written to: /Users/marikhomeriki/code/marikhomeriki/product_review_analysis/CNN_model/assets
